## Как выглядит тестовая выборка

In [1]:
import pandas as pd

sample = pd.read_csv('sample_submission.csv', index_col='Id', sep=',')
sample[:5]

,y
Id,
0,neg
1,neg
2,neg
3,neg
4,neg


In [2]:
import codecs
import re
from pymystem3 import Mystem

test = codecs.open('test.csv', 'r', 'utf_8_sig').read()
test = re.sub(r'[^\w]', ' ', test).lower()    # чистим от знаков
test = test.split('review')[1::2]    # разделяем на список

lemma_test = []
for descr in test:
    lemmas = Mystem().lemmatize(descr)
    lemma_test.append(''.join(lemmas))
test = lemma_test
test[:2]

[' ужасно слабый аккумулятор  это основной минус этот аппарат  разряжаться буквально за пара час при включать wifi и на макс подсветка  например если играть или смотреть видео  следовательно использовать можно только если быть постоянный возможность подзарядиться  качество звук через динамик далеко не на высота наблюдаться незначительный тормоз в некоторый приложение и вообще в меню  очень мало встроенный память  а приложение устанавливаться именно туда  с этот связанный неудобство   нужно постоянно переносить они на карта память  несколько неудобно что нету отдельный кнопка для фото  подумывать купить батарея больший емкость мб что нибудь изменяться    \n',
 ' ценанадежность неубиваемостьдолго держать батарея 4 день стабильно как телефон  3 4 как плеер если  постоянно долбиться в ухо и звонить по пара часо на день  игра и  конечно   смс   в месяц около 200 шт набираться   максимальный время работа 5 день в щадить режим 2 simqwerty рулить  после она набор смс на обычный сенсорник и кно

## Парсим яндекс.маркет и собираем обучающую выборку
Оценка ниже 4 считается негативной

In [4]:
import requests
import bs4
import time
import random
import re
import pandas as pd

reviews = []
rating = []
headers = {'accept': '*/*',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
for n in range(1, 51):    # 50*24 = 1200 отзывов
    time.sleep(random.randint(1, 50))
    session = requests.Session()
    req = session.get('https://market.yandex.ru/catalog--mobilnye-telefony-otzyvy-pokupatelei/54726/list?show-reviews=1&page={}&local-offers-first=0&deliveryincluded=0&onstock=0'.format(n),
                     headers=headers)
    if req.status_code == 200:
        print(n)
    else:
        break
    
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    x = parser.findAll('dl')    # тексты отзывов
    for i, text in enumerate(x[:-2]):
        review = []
        text = re.sub(r'[^\w]', ' ', text.text).lower()    # избавляемся от знаков и приводим к нижнему регистру
        if re.match(r'достоинства', text):
            review.append(text)
            review.append(re.sub(r'[^\w]', ' ', x[i+1].text).lower())
            review.append(re.sub(r'[^\w]', ' ', x[i+2].text).lower())
            review = ' '.join(review)
            reviews.append(review)
        elif not re.match(r'недостатки', text) and not re.match(r'комментарий', text):
            reviews.append(text)
    y = parser.findAll('div', attrs={'class': 'rating__value'})    # оценки
    for label in y[1::2]:
        if int(label.text) > 3:
            rating.append('pos')
        else:
            rating.append('neg')

df = pd.DataFrame({'review': reviews, 'rating': rating})
df.to_csv('reviews.csv', sep=';')

In [3]:
import pandas as pd

df = pd.read_csv('reviews.csv', sep=';', index_col=False)

## Пример отзыва и таблица с разметкой

In [4]:
print(df['review'][0])
df[:5]

  отличный аппарат  продолжение старый добрый линейка  redmi note   экран классный  не amoled  конечно  но ltps   это чуток хорошо чем ips  все работать шустро  слот наконец то тройной  nfc    это вообще просить давно и безответный  современный прошивка  android 9 с переход на 10  камера на отличный уровень  особенно с гугл камера  рано то гугл камера можно быть только  с бубен  поставлять 
   жаль что убирать диод уведомление  и вес мочь быть мало  но это плата за батарея  быть бы вес мало  жаловаться бы на маленький батарея  насчет запись звонок можно поставлять любой кастый на miui и быть вы счастие  причем они бывать и с обновление 
   в целое аппарат не сказать что прямо  огонь   но то что  огонек    это 100500    по цена качество рядом ничто не лежать  если что то и лежать  то  кто читать и знать  писать  быть очень интересно  спасибо  что прочитывать отзыв до конец 



,review,rating
0,отличный аппарат продолжение старый добрый ...,pos
1,большой экран аккумулятор \n не увидеть ...,pos
2,купить он спонтанно зайти на маркет увидеть...,pos
3,1 скорость и плавность работа у телефон 6 ...,pos
4,1 брать ради камера не мочь сказать что он...,pos


In [5]:
df2 = pd.DataFrame({'review': [x for i, x in enumerate(df['review']) if df['rating'][i] == 'neg'] * 10,
                   'rating': ['neg'] * 880})
df = df.append(df2)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
# Tfidf Vectorizer
ppl = Pipeline([
    ('vectorizer', CountVectorizer(stop_words=stop_words, ngram_range=(1, 2))),
    ('classifier', LogisticRegression(solver='liblinear')),
])

print('Качество на обучении:', cross_val_score(ppl, df['review'], df['rating'], scoring='accuracy', cv=5).mean())

Качество на обучении: 0.9842169442169443


In [7]:
ppl.fit(df['review'], df['rating'])
pred = ppl.predict(test)
pred = pd.DataFrame({'Id': list(range(100)), 'y': pred})
pred.to_csv('pred.csv', sep=',', index=False)

In [8]:
ppl.predict(test)

array(['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos'], dtype=object)